In [99]:
import numpy as np
import joblib

In [100]:

class MaRegressionLasso:
        def __init__(self, alpha=0.1, lr=0.01, n_iter=1000):
            self.alpha = alpha
            self.lr = lr
            self.n_iter = n_iter
            self.w = None
            self.b = 0

alpha : régularisation L1 (faxteur de pénalisation)
lr: facteur d'apprentissage (learning rate dans la literature habituelle du python)
n_iter : nombre d'itérations

In [101]:
    def initialisation_poids_biais(self, n_features):
        self.w = np.zeros(n_features)
        self.b = 0

In [102]:
MaRegressionLasso.initialisation_poids_biais=initialisation_poids_biais

La necessite de definir cette foncion nous permet de ne pas trimballer des expression mathematique dans tout notre programme ameliorer la maintenabilite de notre mODEL.

In [103]:
    def predict_valeur_reelle(self, X):
        return np.dot(X, self.w) + self.b

In [104]:
MaRegressionLasso.predict_valeur_reelle=predict_valeur_reelle

 Calcule la fonction de cout avec régularisation L1

In [105]:
    def fonction_cout(self, X, y):
        m = len(y)
        y_pred = self.predict_valeur_reelle(X)
        mse = (1 / (2 * m)) * np.sum((y - y_pred) ** 2)
        penalte_l1 = self.alpha * np.sum(np.abs(self.w))
        return mse + penalte_l1

In [106]:
MaRegressionLasso.fonction_cout=fonction_cout

Calcule les gradients pour les poids et le biais.On utilise ici le sous-gradient pour la pénalisation L1. Il est tres important de comprendre ici la notion de sous-gradient. 
La regresion lasso minimise la fonction suivante : 

![La fonction coût](fonction_cout.png),
Le calcul du gradient de la valeur absolue est 
![La derive de la valeur absolue](derive_valeur_absolue.png)

Le sous-gradient est une généralisation du concept de dérivée pour les fonctions non différentiables.Même si la pente exacte n’existe pas à un point, on peut trouver un ensemble de pentes valides qui "touchent" la fonction sans la dépasser.pour une fonction convexe f(x),un sous-gradientg en x0  est  un vecteur tel que:
![sous gradient](sous_gradient.png).
Le Lasso met une barre aimantée au zéro :quand un poids devient petit, il “colle” à zéro et n’en bouge plus.Cet “aimant” vient directement du sous-gradient. Tandis que Ridge met des “ressorts” qui tirent doucement les poids vers zéro.

In [107]:
    def Descente_gradients(self, X, y):
        m = len(y)
        y_pred = self.predict_valeur_relle(X)
        dw1 = -(1 / m) * np.dot(X.T, (y - y_pred))
        db = -(1 / m) * np.sum(y - y_pred)
    #sous gradient  de valeur absolu de w
        dw2 = self.alpha * np.sign(self.w)
        dw=dw1+dw2
    
        return dw, db

In [108]:
MaRegressionLasso.Descente_gradients=Descente_gradients

 Nous avons decide d afficher le cout toutes les 100 itérations pour suivre la progression de l apprentissage du modele

In [109]:
    def entrainement(self, X, y):
        X = np.array(X)
        y = np.array(y)
        n_samples, n_features = X.shape
        self.initialisation_poids_biais(n_features)
    
        for i in range(self.n_iter):
            dw, db = self.Descente_gradients(X, y)
            self.w -= self.lr * dw
            self.b -= self.lr * db
            if i % 100 == 0:
                cout= self.fonction_cout(X, y)
                print(f"Iteration {i:04d} | Cout: {cout:.4f}")

In [110]:
MaRegressionLasso.entrainement=entrainement

In [111]:
    def prediction(self, X):
        X = np.array(X)
        return self.predict_valeur_reelle(X)

In [112]:
MaRegressionLasso.prediction=prediction

In [113]:
model=MaRegressionLasso()

In [114]:
#joblib.dump(model,filename="model_lasso.joblib",compress=0) 
dir(model)

['Descente_gradients',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__firstlineno__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__static_attributes__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'alpha',
 'b',
 'entrainement',
 'fonction_cout',
 'initialisation_poids_biais',
 'lr',
 'n_iter',
 'predict_valeur_reelle',
 'prediction',
 'w']

In [116]:
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
#from Regression_Lasso import MaRegressionLasso 

# Jeu de données synthétique
X, y = make_regression(n_samples=500, n_features=5, noise=10, random_state=42)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modèle Lasso maison
model = MaRegressionLasso(alpha=0.05, lr=0.001, n_iter=2000)
model.entrainement(X_train, y_train)

# Évaluation
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("\n✅ Résultats finaux :")
print(f"MSE = {mse:.2f}")
print(f"R² = {r2:.3f}")
print(f"Poids appris : {model.w}")


AttributeError: 'MaRegressionLasso' object has no attribute 'predict_valeur_relle'